In [4]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
debug = 1
import numpy as np
import pandas
import os
#from SciServer import CasJobs
#from pprint import pprint

pandas.set_option('display.max_colwidth', -1)

basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/'
rawdatadir = basedir + 'rawdata/'
metadir = basedir + 'metadata/xls_temp/'

datadir = basedir + 'data/'
errordir = basedir + 'error/'
vardir = basedir + 'variables/'
geodir = basedir + 'geography/'

for thisdir in [datadir, errordir, vardir, geodir]:
    if not(os.path.exists(thisdir)):
        os.makedirs(thisdir)
print('ok')

ok


In [ ]:
# category can be one of: demographics, ancestry, residence, transportation, household, income, employment, housing, qa
category = 'housing'

states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

data_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()
error_df = pandas.DataFrame()

# 1-6, 139-150
want_sequences = []
for i in range(1,2):
    want_sequences.append(i)

if (category == 'demographics'):
    for i in range(2,7):
        want_sequences.append(i)
elif (category == 'ancestry'):
    for i in range(7,19):
        want_sequences.append(i)
elif (category == 'residence'):
    for i in range(19,29):
        want_sequences.append(i)
elif (category == 'transportation'):
    for i in range(29,45):
        want_sequences.append(i)
elif (category == 'household'):
    for i in range(45,62):
        want_sequences.append(i)
elif (category == 'income'):
    for i in range(62,92):
        want_sequences.append(i)
elif (category == 'employment'):
    for i in range(92,139):
        want_sequences.append(i)
elif (category == 'housing'):
    for i in range(139,162):
        want_sequences.append(i)
elif (category == 'qa'):
    for i in range(162,167):
        want_sequences.append(i)
else:
    print('Category {0:} not found!'.format(category))
    
for i in want_sequences:
    if (debug >= 1):
#        if ((np.mod(i, 10) == 0) | (i == len(want_sequences))):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_data_df = pandas.DataFrame()
    this_seq_metadata_filename = metadir + 'seq{0:.0f}.xlsx'.format(i)
#    this_seq_metadata_filename = '/home/idies/workspace/raddick_acs_data/metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_in_cols_df = this_seq_metadata_in_cols_df.dropna(axis=1)
    
    if (i == 1):
        this_seq_metadata_in_cols_df.columns = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','B00001_001_UW','B00001_002_UW']
    else:
        this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    if (i == 1):
        this_seq_metadata_df['variable'] = this_seq_metadata_in_cols_df.columns
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1] 
    else:
        this_seq_metadata_df['sequence_number'] = i

    this_seq_data_df = pandas.DataFrame()
    for onestate in states:
        statefilename = rawdatadir + 'e20171{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8')
        this_seq_data_df = this_seq_data_df.append(onestate_df)

    this_seq_data_df.columns = this_seq_metadata_in_cols_df.columns
    
    this_seq_data_df = this_seq_data_df.drop(['SEQUENCE'], axis=1)

    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_data_df = this_seq_data_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    data_df = pandas.concat((data_df, this_seq_data_df), axis=1)

if (debug >= 2):
    print('Creating pseudo geoid...')
data_df['PSEUDO_GEOID'] = data_df['STUSAB'].str.upper() + data_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))

if (debug >= 2):
    print('Indexing...')
data_df = data_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geo_metadata_in_cols_df = pandas.read_excel(metadir + '2016_SFGeoFileTemplate_maybe_2017.xls', header=None, encoding='utf-8')
geo_metadata_in_cols_df = geo_metadata_in_cols_df.dropna(axis=1)
geo_metadata_in_cols_df.columns = geo_metadata_in_cols_df.loc[0]
geo_metadata_in_cols_df.index = ['variable','description']

geo_metadata_df = geo_metadata_in_cols_df.T
geo_metadata_df.columns = ['variable','description']

geo_df = pandas.DataFrame()
for onestate in states:
    try:
        this_geo_df = pandas.read_csv(rawdatadir + 'g20171{0:}.csv'.format(onestate), low_memory=False, encoding='iso-8859-1', header=None)
    except UnicodeDecodeError:
        print(rawdatadir + 'g20171{0:}.csv not found!'.format(onestate))
    geo_df = pandas.concat((geo_df, this_geo_df), axis=0)

geo_df.columns = geo_metadata_in_cols_df.columns

geo_df['PSEUDO_GEOID'] = geo_df['STUSAB'].str.upper() + geo_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))

geo_df = geo_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')

data_df = data_df.join(geo_df[['GEOID','NAME']])

if (debug >= 2):
    print('Resetting index...')

metadata_df.loc['GEOID'] = ['GEOID', 'Geography identifier (also functions as index of data tables)', 0]
metadata_df.loc['NAME'] = ['NAME', 'Name of geography region', 0]

data_df = data_df.set_index('GEOID', drop=False)
data_df.index.name = ''

metadata_df = metadata_df.set_index('variable', drop=False)

if (debug >= 1):
    print('Writing variables file...')

if not (os.path.exists(vardir)):
    os.makedirs(vardir)
if not (os.path.exists(geodir)):
    os.makedirs(geodir)

metadata_df.to_csv(vardir + 'variables_acs2017_{0:}.csv'.format(category), encoding='utf-8')
geo_df.to_csv(geodir + 'geo_acs2017.csv', encoding='utf-8')
geo_metadata_df.to_csv(geodir + 'geo_variables_acs2017.csv', encoding='utf-8')

if (debug >= 1):
    print('Writing data (estimates) file...')

    
if not (os.path.exists(datadir)):
    os.makedirs(datadir)
    
data_df.to_csv(datadir + 'data_acs2017_{0:}.csv'.format(category), encoding='utf-8')

if (debug >= 1):
    print('Done!')

Importing data for sequence 1...
Importing data for sequence 139...
Importing data for sequence 140...
Importing data for sequence 141...
Importing data for sequence 142...
Importing data for sequence 143...
Importing data for sequence 144...
Importing data for sequence 145...
Importing data for sequence 146...
Importing data for sequence 147...
Importing data for sequence 148...
Importing data for sequence 149...
Importing data for sequence 150...
Importing data for sequence 151...
Importing data for sequence 152...
Importing data for sequence 153...
Importing data for sequence 154...
Importing data for sequence 155...
Importing data for sequence 156...
Importing data for sequence 157...
Importing data for sequence 158...
Importing data for sequence 159...
Importing data for sequence 160...
Importing data for sequence 161...


In [ ]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd

# category can be one of: demographics, ancestry, residence, transportation, household, income, employment, housing, qa
category = 'qa'

basedir = '/home/idies/workspace/persistent/censusdata/'
rawdatadir = basedir + 'rawdata/'
metadir = basedir + 'metadata/Templates/'

datadir = basedir + 'data/'
errordir = basedir + 'error/'
vardir = basedir + 'variables/'
geodir = basedir + 'geography/'

debug = 1
import numpy as np
import pandas
import os
#from SciServer import CasJobs
#from pprint import pprint

pandas.set_option('display.max_colwidth', -1)
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

data_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()
error_df = pandas.DataFrame()

# 1-6, 139-150
want_sequences = []
for i in range(1,2):
    want_sequences.append(i)

if (category == 'demographics'):
    for i in range(2,7):
        want_sequences.append(i)
elif (category == 'ancestry'):
    for i in range(7,19):
        want_sequences.append(i)
elif (category == 'residence'):
    for i in range(19,29):
        want_sequences.append(i)
elif (category == 'transportation'):
    for i in range(29,45):
        want_sequences.append(i)
elif (category == 'household'):
    for i in range(45,62):
        want_sequences.append(i)
elif (category == 'income'):
    for i in range(62,92):
        want_sequences.append(i)
elif (category == 'employment'):
    for i in range(92,139):
        want_sequences.append(i)
elif (category == 'housing'):
    for i in range(139,162):
        want_sequences.append(i)
elif (category == 'qa'):
    for i in range(162,167):
        want_sequences.append(i)
else:
    print('Category {0:} not found!'.format(category))
    
for i in want_sequences:
    if (debug >= 1):
#        if ((np.mod(i, 10) == 0) | (i == len(want_sequences))):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_error_df = pandas.DataFrame()
    this_seq_metadata_filename = metadir + 'seq{0:.0f}.xlsx'.format(i)
#    this_seq_metadata_filename = '/home/idies/workspace/raddick_acs_data/metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_in_cols_df = pandas.read_excel(this_seq_metadata_filename, header=None, encoding='utf-8')
    this_seq_metadata_in_cols_df = this_seq_metadata_in_cols_df.dropna(axis=1)
    
    if (i == 1):
        this_seq_metadata_in_cols_df.columns = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','B00001_001_UW','B00001_002_UW']
    else:
        this_seq_metadata_in_cols_df.columns = this_seq_metadata_in_cols_df.loc[0]

    this_seq_metadata_df = this_seq_metadata_in_cols_df.T
    this_seq_metadata_df.columns = ['variable', 'description']
    if (i == 1):
        this_seq_metadata_df['variable'] = this_seq_metadata_in_cols_df.columns
        this_seq_metadata_df['sequence_number'] = [0,0,0,0,0,0,1,1] 
    else:
        this_seq_metadata_df['sequence_number'] = i

    for onestate in states:
        statefilename = rawdatadir + 'e20171{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None, sep=',', encoding='utf-8')
        this_seq_error_df = this_seq_error_df.append(onestate_df)

    this_seq_error_df.columns = this_seq_metadata_in_cols_df.columns
    
    this_seq_error_df = this_seq_error_df.drop(['SEQUENCE'], axis=1)

    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB', 'SEQUENCE', 'CHARITER','LOGRECNO'], axis=0)
        this_seq_error_df = this_seq_error_df.drop(['FILEID','FILETYPE','STUSAB', 'CHARITER', 'LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=0)
    error_df = pandas.concat((error_df, this_seq_error_df), axis=1)

if (debug >= 2):
    print('Creating pseudo geoid...')
error_df['PSEUDO_GEOID'] = error_df['STUSAB'].str.upper() + error_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))

if (debug >= 2):
    print('Indexing...')
error_df = error_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geo_metadata_in_cols_df = pandas.read_excel(metadir + '2016_SFGeoFileTemplate_maybe_2017.xls', header=None, encoding='utf-8')
geo_metadata_in_cols_df = geo_metadata_in_cols_df.dropna(axis=1)
geo_metadata_in_cols_df.columns = geo_metadata_in_cols_df.loc[0]
geo_metadata_in_cols_df.index = ['variable','description']

geo_metadata_df = geo_metadata_in_cols_df.T
geo_metadata_df.columns = ['variable','description']

geo_df = pandas.DataFrame()
for onestate in states:
    try:
        this_geo_df = pandas.read_csv(rawdatadir + 'g20171{0:}.csv'.format(onestate), low_memory=False, encoding='iso-8859-1', header=None)
    except UnicodeDecodeError:
        print(rawdatadir + 'g20171{0:}.csv not found!'.format(onestate))
    geo_df = pandas.concat((geo_df, this_geo_df), axis=0)

geo_df.columns = geo_metadata_in_cols_df.columns

geo_df['PSEUDO_GEOID'] = geo_df['STUSAB'].str.upper() + geo_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))

geo_df = geo_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')

error_df = error_df.join(geo_df[['GEOID','NAME']])

if (debug >= 2):
    print('Resetting index...')

metadata_df.loc['GEOID'] = ['GEOID', 'Geography identifier (also functions as index of data tables)', 0]
metadata_df.loc['NAME'] = ['NAME', 'Name of geography region', 0]

error_df = error_df.set_index('GEOID', drop=False)
error_df.index.name = ''

metadata_df = metadata_df.set_index('variable', drop=False)

if (debug >= 1):
    print('Writing variables file...')

if not (os.path.exists(vardir)):
    os.makedirs(vardir)
if not (os.path.exists(geodir)):
    os.makedirs(geodir)

metadata_df.to_csv(vardir + 'variables_acs2017_{0:}.csv'.format(category), encoding='utf-8')
geo_df.to_csv(geodir + 'geo_acs2016.csv', encoding='utf-8')
geo_metadata_df.to_csv(geodir + 'geo_variables_acs2017.csv', encoding='utf-8')

if (debug >= 1):
    print('Writing margins of error file...')

    
if not (os.path.exists(errordir)):
    os.makedirs(errordir)
    
error_df.to_csv(errordir + 'error_acs2017_{0:}.csv'.format(category), encoding='utf-8')

if (debug >= 1):
    print('Done!')

In [ ]:

# reorder columns
#new_column_list = []
#new_column_list.append(geo_metadata_df.columns[len(geo_metadata_df.columns)-5])
#for i in range(0,len(geo_metadata_df.columns)-5):
#    new_column_list.append(geo_metadata_df.columns[i])
#for j in range(len(geo_metadata_df.columns)-4, len(geo_metadata_df.columns)):
#    new_column_list.append(geo_metadata_df.columns[j])
#geo_metadata_df.columns = new_column_list

#new_column_list = []
#new_column_list.append(geo_df.columns[len(geo_df.columns)-5])
#for i in range(0,len(geo_df.columns)-5):
#    new_column_list.append(geo_df.columns[i])
#for j in range(len(geo_df.columns)-4, len(geo_df.columns)):
#    new_column_list.append(geo_df.columns[j])



In [ ]:
#print('Females under age 5 in Baltimore County: {0:,.0f} with margin of error {1:,.0f}'.format(data_df['B01001_027'].loc['05000US24510'], error_df['B01001_027'].loc['05000US24510']))
#data_df['B01001_027'].loc['05000US24510']
#error_df['B01001_027'].loc['05000US24510']